# Flame

In order to download the models we need to look at: https://flame.is.tue.mpg.de/download.php
This could be usefull if onw want's to look how to load the FLAME model from the SMPL loader: https://github.com/Rubikplayer/flame-fitting/blob/master/smpl_webuser/serialization.py#L117
Useful utils if one needs to transform the chumpy format into nupy or torch: https://github.com/vchoutas/smplx/blob/main/smplx/utils.py

NOTE: That if one want't to unpickle old python=2.x numpy code, we need to use the encoding="latin1". For more information please refere to: https://docs.python.org/3/library/pickle.html


In [ ]:
import numpy as np
from lib.utils.loader import load_flame
from lib.model.flame import FLAME

# https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/flame_pytorch/flame.py
data_dir = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove"
flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_dict = load_flame(flame_dir)
flame_model = FLAME(flame_dir=flame_dir, data_dir=data_dir)
print("FLAME keys:")
print(list(flame_dict.keys()))
print()

# This is the linear blend skinning (LBS) with corrective blendshapes with N=5023 and
# K=4 joint (neck, jaw, and eyeballs (left, right))
bs_style = flame_dict["bs_style"]
print("bs_style:", bs_style)
bs_type = flame_dict["bs_type"]
print("bs_type:", bs_type)

# this is the template mesh, e.g. T bar in the "zero pose"
v_template = flame_dict["v_template"]
print("v_template:", v_template.shape)
for i in range(3):
    d = v_template[:, i].max() - v_template[:, i].min()
    s = ["x", "y", "z"][i]
    print(f"{s}-delta in meter {d:.2}m")

# those are used in the pytorch flame example
f = flame_dict["f"]
print("f:", f.shape)

# shape (beta); note that the dimension is (5023, 3, 400)
# where the first 300 are for the shape params and the last 400 for the expression
# params, but the matrix is shared
shapedirs = flame_dict["shapedirs"]
print("shapedirs:", shapedirs.shape)

# pose (theta)
posedirs = flame_dict["posedirs"]
print("posedirs:", posedirs.shape)

# is this the expressions? (psi)
weights = flame_dict["weights"]  # lbs := linear blend shapes
print("weights:", weights.shape)

# Linear smoothed by skinning function(T, J, theta, W).
# Blendweights W (KxN) are J_regressor
J_regressor = flame_dict["J_regressor"]
print("J_regressor:", J_regressor.shape)

# J are the joints that the vertices of T are rotated
J = flame_dict["J"]
print("J:", J.shape)

kintree_table = flame_dict["kintree_table"]
print("kintree_table:", kintree_table.shape)

# Falme Landmarks

The landmark file defines the barycentric embedding of 105 points of the Mediapipe mesh in the surface of FLAME.
In consists of three arrays: lmk_face_idx, lmk_b_coords, and landmark_indices.

- lmk_face_idx contains for every landmark the index of the FLAME triangle which each landmark is embedded into
- lmk_b_coords are the barycentric weights for each vertex of the triangles
- landmark_indices are the indices of the vertices of the Mediapipe mesh


In [ ]:
from lib.utils.loader import load_static_landmark_embedding

flame_landmarks = load_static_landmark_embedding(flame_dir, "pt")
print(list(flame_landmarks.keys()))
print()

print("lmk_face_idx:")
print(flame_landmarks["lm_face_idx"][:5])
print(flame_landmarks["lm_face_idx"].min())
print(flame_landmarks["lm_face_idx"].max())
print(flame_landmarks["lm_face_idx"].shape)
print()

print("lmk_b_coords:")
print(flame_landmarks["lm_bary_coords"][:5])
print(flame_landmarks["lm_bary_coords"].min())
print(flame_landmarks["lm_bary_coords"].max())
print(flame_landmarks["lm_bary_coords"].shape)
print()

print("landmark_indices:")
print(flame_landmarks["lm_mediapipe_idx"][:5])
print(flame_landmarks["lm_mediapipe_idx"].min())
print(flame_landmarks["lm_mediapipe_idx"].max())
print(flame_landmarks["lm_mediapipe_idx"].shape)

# FLAME Mask

Dictionary with vertex indices for different masks for the publicly available FLAME head model (https://flame.is.tue.mpg.de/).
See the gif for a visualization of all masks.

Those are the vertices.


In [ ]:
from lib.utils.loader import load_flame_masks
import torch

flame_masks = load_flame_masks(flame_dir)
print(list(flame_masks.keys()))
print()

print("face:")
print(flame_masks["face"][:5])
print(flame_masks["face"].min())
print(flame_masks["face"].max())
print(flame_masks["face"].shape)

faces_mask = torch.tensor(flame_masks["face"])

In [ ]:
flame_masks["face"]

# Pytorch3D Rasterizer

We want to implement our own rasterizer, hence we can look how pytorch metric is doing it:
from pytorch3d.renderer.mesh import rasterize_meshes
Or we can implmenet it, for reference here:
https://www.scratchapixel.com/lessons/3d-basic-rendering/rasterization-practical-implementation/rasterization-stage.html

Go over the rasterization:
https://www.scratchapixel.com/lessons/3d-basic-rendering/rasterization-practical-implementation/overview-rasterization-algorithm.html


In [ ]:
import torch
from lib.model.flame import FLAME
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.utils.loader import load_intrinsics
import matplotlib.pyplot as plt

flame_dir = "/home/borth/GuidedResearch/checkpoints/flame2023"
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"
flame = FLAME(
    flame_dir=flame_dir,
    data_dir=data_dir,
    optimize_shapes=1,
    optimize_frames=100,
)
flame.init_params_dphm()
K = load_intrinsics(data_dir, return_tensor="pt")
camera = Camera(K=K, width=1920, height=1080, near=0.01, far=100, scale=1.0)
rasterizer = Rasterizer(width=camera.width, height=camera.height)
flame.init_renderer(camera=camera, rasterizer=rasterizer)
flame = flame.to("cuda")

In [ ]:
flame.shape_params.weight.device

In [ ]:
shape_idx = torch.tensor([0] * 5).to("cuda")
frame_idx = torch.tensor([0] * 5).to("cuda")
batch = {"shape_idx": shape_idx, "frame_idx": frame_idx}
out = flame.optimization_step(batch)

In [ ]:
import time
shape_idx = torch.tensor([0] * 5).to("cuda")
frame_idx = torch.tensor([0] * 5).to("cuda")
batch = {"shape_idx": shape_idx, "frame_idx": frame_idx}
model = flame.model_step(batch)
n = 100
mt = 0
rt = 0
for _ in range(n):
    # s = time.time()
    # model = flame.model_step(batch)
    # e = time.time() 
    # mt += e - s

    s = time.time()
    render = flame.render_step(model)
    e = time.time()
    rt += e - s


print("Model Step:")
print(f"Total time: {mt}s")
print(f"Mean time: {(mt / n) * 1000} ms")

print("Render Step:")
print(f"Total time: {rt}s")
print(f"Mean time: {(rt / n) * 1000} ms")


In [ ]:
import torch
print("Model Step:")
print(f"Total time: {mt}s")
print(f"Mean time: {(mt / n) * 1000} ms")

scale = 0.5
height = int(1080 * scale)
width = int(1920 * scale)

near = 1.0
far = 100
fx = 914.4150 * scale
fy = 914.0300 * scale
cx = 959.5980 * scale
cy = 547.2020 * scale
A = near + far
B = near * far

l = 0
r = width
b = height
t = 0
# l = -width/2
# r = width/2
# b = -height/2
# t = height

tx = -(r + l) / (r - l)
ty = -(t + b) / (t - b)
tz = -(far + near) / (far - near)

Persp = torch.tensor(
    [
        [fx, 0.0, -cx, 0.0],
        [0.0, -fy, -cy, 0.0],
        [0.0, 0.0, A, B],
        [0.0, 0.0, -1.0, 0.0],
    ]
)

NDC = torch.tensor(
    [
        [2 / (r - l), 0.0, 0.0, tx],
        [0.0, 2 / (t - b), 0.0, ty],
        [0.0, 0.0, -2 / (far - near), tz],
        [0.0, 0.0, 0.0, 1.0],
    ]
)

Proj = NDC @ Persp
Proj

In [ ]:
from lib.model.flame import FLAME
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.utils.loader import load_intrinsics
import matplotlib.pyplot as plt

flame_dir = "/home/borth/GuidedResearch/checkpoints/flame2023_no_jaw"
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"

flame = FLAME(
    flame_dir,
    data_dir=data_dir,
    vertices_mask="full",
).to("cuda")
flame.init_params(
    global_pose=[0, 0, 0],
    transl=[0, 0, -0.5],
    neck_pose=[0.2, 0.2, 0.2],
    jaw_pose=[0.3, 0.2, 0.7],
    eye_pose=[2, 0.5, 0.1, 0.0, 0.0, 0.0],
)
vertices, landmarks = flame()
K = load_intrinsics(data_dir, return_tensor="pt")
camera = Camera(fov_y=45, width=1920, height=1080, near=0.01, far=100, scale=1.0)
rasterizer = Rasterizer(width=camera.width, height=camera.height)
renderer = Renderer(camera=camera, rasterizer=rasterizer)
render = renderer.render_full(vertices, flame.faces)
plt.imshow(render["normal_image"][0].detach().cpu().numpy())

In [ ]:
renderer = flame.renderer(height=100, width=100, fov=fov, near=near, far=far)
render = renderer.render_full(vertices, flame.masked_faces(vertices))
plt.imshow(render["normal_image"][0].detach().cpu().numpy())

In [ ]:
from lib.renderer.camera import FoVCamera

width = 600
height = 600
renderer.rasterizer.glctx.width = width
renderer.rasterizer.glctx.height = height
renderer.camera = FoVCamera(
    fov, aspect=(width / height), near=1.0, far=100.0, device="cuda"
)
render = renderer.render_full(
    vertices.to("cuda"), flame.masked_faces(vertices).to("cuda")
)
plt.imshow(render["normal_image"][0].detach().cpu().numpy())

In [ ]:
from lib.rasterizer import Rasterizer
from lib.renderer.camera import FoVCamera

width = 800
height = 800
fov = 45
near = 0.3
far = 100

renderer = flame.renderer(fov=fov, width=width, height=height, near=near, far=far)
# homo_vertices = convert_to_homo_coords(vertices)
# homo_clip_vertices = renderer.camera.transfrom(vertices)  # (B, V, 4)
# (fragments.pix_to_face == -1).sum()
# homo_clip_vertices
# camera.M @ homo_vertices[0, 0]

render = renderer.render_full(vertices, flame.faces[:1])
plt.imshow(render["normal_image"][0].detach().cpu().numpy())
# (fragments.pix_to_face != -1).sum(), clip_vertices[:, :, :3].max(),clip_vertices[:, :, :3].min()

In [ ]:
clip_vertices = clip_vertices[:, :4, :]
clip_vertices

In [ ]:
flame.faces[:1].shape

In [ ]:
camera.M

In [ ]:
from lib.renderer.camera import FoVCamera

import numpy as np

camera = FoVCamera(fov=20, aspect=1.0, near=1.0, far=100)
clip_v = camera.transfrom(vertices)
clip_v[:, :, 0] /= clip_v[:, :, 3]
clip_v[:, :, 1] /= clip_v[:, :, 3]
clip_v[:, :, 2] /= clip_v[:, :, 3]
points = clip_v[0, :, :3].detach().cpu().numpy().astype(np.float32)

color = [255, 0, 0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
color = [np.array(color, dtype=np.uint8)] * points.shape[0]
pcd.colors = o3d.utility.Vector3dVector(np.stack(color))

o3d.visualization.draw_plotly([pcd])

In [ ]:
v = vertices.detach().cpu().numpy().astype(np.float32)[0]
color = [255, 0, 0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(v)
color = [np.array(color, dtype=np.uint8)] * points.shape[0]
pcd.colors = o3d.utility.Vector3dVector(np.stack(color))
o3d.visualization.draw_plotly([pcd])

In [ ]:
from lib.renderer.camera import FoVCamera

camera = FoVCamera(fov=45, aspect=1.0, near=1.0, far=100)
pcamera = torch.tensor([0.1, 0.2, -1.2, 1.0])
pclip = camera.M @ pcamera
print(pclip)
pndc = pclip.clone()
pndc[0] /= pndc[3]
pndc[1] /= pndc[3]
pndc[2] /= pndc[3]
pndc[3] /= pndc[3]
print(pndc)
pcamera_inv = camera.M.inverse() @ pclip
print(pcamera_inv)

In [ ]:
renderer.camera.transfrom(vertices)

In [ ]:
hv = convert_to_homo_coords(vertices)[0, 0]
print(hv)
renderer.camera.M @ hv.to("cpu")

In [ ]:
torch.matmul(
    renderer.camera.M.to("cpu"),
    convert_to_homo_coords(vertices).to("cpu").transpose(1, 2),
).transpose(1, 2)

In [ ]:
renderer.camera.M.to("cpu").expand(10, -1, -1)

In [ ]:
from lib.utils.mesh import vertex_normals

renderer = flame.renderer()
normal, mask = renderer.render_normal(vertices, flame.faces)
normal_image = renderer.normal_to_normal_image(normal, mask)
plt.imshow(normal_image[0].detach().cpu().numpy())
plt.show()

In [ ]:
import numpy as np
import open3d as o3d

color = [255, 0, 0]
points = vertices.detach().cpu().numpy().astype(np.float64)[0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
color = [np.array(color, dtype=np.uint8)] * points.shape[0]
pcd.colors = o3d.utility.Vector3dVector(np.stack(color))
o3d.visualization.draw_plotly([pcd])

In [ ]:
shape = list(vertices.shape)
shape[-1] = shape[-1] + 1
homo_vertices = torch.ones(shape, device=vertices.device)
homo_vertices[:, :, :3] = vertices

In [ ]:
from lib.renderer.camera import camera2pixel

# fix
K = flame.K
vp = camera2pixel(landmarks[0], K[0, 0], K[1, 1], K[0, 2], K[1, 2])
# vp[flame.faces][:, :, :2]
# vp.shape

In [ ]:
K

In [ ]:
from lib.renderer.camera import camera2normal

point, mask = renderer.render_point(vertices, flame.faces)
n, m = camera2normal(point)
normal_image = renderer.normal_to_normal_image(n, m)
plt.imshow(normal_image.detach().cpu().numpy()[0])

In [ ]:
from lib.utils.loader import load_color

color_image = load_color(data_dir=data_dir, idx=0, return_tensor="pt")
image = renderer.render_color_image(
    vertices, flame_model.faces, color_image, faces_mask, True
)
plt.imshow(image.detach().cpu().numpy())
plt.show()

In [ ]:
normal = renderer.render_normal_image(vertices, flame_model.faces, faces_mask)
plt.imshow(normal.detach().cpu().numpy())
plt.show()

In [ ]:
depth = renderer.render_shader_image(vertices, flame_model.faces, faces_mask)
plt.imshow(depth.detach().cpu().numpy())
plt.show()

In [ ]:
import matplotlib.pyplot as plt

path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023_no_jaw/FLAME_texture.npz"
albedo = np.load(path)
albedo_faces = albedo["vt"][albedo["ft"]]
albedo_map = albedo["mean"].astype(np.uint8)
plt.imshow(albedo_map[:, :, ::-1])
# albedo_faces.shape

# Open3D Point Cloud


In [ ]:
import open3d as o3d
import numpy as np
from pathlib import Path
from lib.utils.loader import load_points_3d

C = 5023
red = [np.array([255, 0, 0], dtype=np.uint8)] * C
red = o3d.utility.Vector3dVector(np.stack(red))
green = [np.array([0, 255, 0], dtype=np.uint8)] * C
green = o3d.utility.Vector3dVector(np.stack(green))
blue = [np.array([0, 0, 255], dtype=np.uint8)] * C
blue = o3d.utility.Vector3dVector(np.stack(blue))

vertices = np.load("temp/vertices.npy")
pcd_flame = o3d.geometry.PointCloud()
pcd_flame.points = o3d.utility.Vector3dVector(vertices)
pcd_flame.colors = blue

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
points = load_points_3d(data_dir=data_dir, idx=0)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = red

o3d.visualization.draw_plotly([pcd, pcd_flame])

# Albedo Diffuse

This describes how to convert from BFM to FLAME:
https://github.com/TimoBolkart/BFM_to_FLAME


In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023_no_jaw/albedoModel2020_FLAME_albedoPart.npz"
albedo = np.load(path)
print(list(albedo.keys()))
print(f"{albedo['vt'].shape=}")
print(f"{albedo['vt'].min()=}")
print(f"{albedo['vt'].max()=}")
print(f"{albedo['ft'].shape=}")
print(f"{albedo['ft'].min()=}")
print(f"{albedo['ft'].max()=}")
print(f"{albedo['specPC'].shape=}")
print(f"{albedo['PC'].shape=}")
print(f"{albedo['specMU'].shape=}")
print(f"{albedo['MU'].shape=}")

In [ ]:
albedo["ft"]

In [ ]:
albedo["vt"].shape[0] - 5023

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(albedo["MU"])

# Other Albedo


In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023_no_jaw/FLAME_texture.npz"
albedo = np.load(path)
albedo_faces = albedo["vt"][albedo["ft"]]
albedo_map = albedo["mean"]

list(albedo.keys())
print(f"{albedo['vt'].shape=}")
print(f"{albedo['vt'].min()=}")
print(f"{albedo['vt'].max()=}")
print(f"{albedo['ft'].shape=}")
print(f"{albedo['ft'].min()=}")
print(f"{albedo['ft'].max()=}")
print(f"{albedo['tex_dir'].shape=}")
print(f"{albedo['mean'].shape=}")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(albedo["mean"].astype(int))

In [ ]:
flame_model

In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023/FLAME_albedo_from_BFM.npz"
albedo = np.load(path)
print(list(albedo.keys()))
albedo["PC"].shape

In [ ]:
from pytorch3d.io import load_obj

verts, faces, aux = load_obj(
    "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023/head_template.obj"
)
aux.verts_uvs.shape

In [ ]:
faces.verts_idx.shape

In [ ]:
verts_uvs

In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/logs/optimize/runs/2024-04-25_17-49-55"
i = 650
vertices = np.load(f"{path}/pcd_vertices/000_{i:05}.npz")